In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d

### Making surfboard Cross Section

In [ ]:

def surfboard_outline(width, thickness, num_points=1000):
    """
    Generate a smooth closed curve representing the outline of a surfboard cross-section using a cubic spline.

    Parameters:
    width (float): Width of the surfboard.
    thickness (float): Thickness of the surfboard.
    bottom_curve (float): Curvature of the bottom half of the surfboard outline.
    num_points (int): Number of points to generate on the curve. Defaults to 100.

    Returns:
    x (numpy.ndarray): x-coordinates of the surfboard outline.
    y (numpy.ndarray): y-coordinates of the surfboard outline.
    """
    # Define control points for Bezier curve
    p0 = [0, thickness/2]
    p2 = [0, -thickness/2]
    p1 = [width/2, (p0[1]+p2[1])/2]
    p3 = [-width/2, (p0[1]+p2[1])/2]
    print(f'Effective thickness is {p0[1]+abs(p2[1])}')
    # Create array of control points
    control_points = np.array([p2, p3, p0, p1, p2])
    plt.scatter(p0[0],p0[1],color='black')
    plt.scatter(p1[0],p1[1],color='blue')
    plt.scatter(p2[0],p2[1],color='orange')
    plt.scatter(p3[0],p3[1],color='green')

    # Generate evenly spaced parameter values for the spline
    t = np.linspace(0, 1, 5)

    # Generate cubic spline interpolation function
    spline_func = interp1d(t, control_points, kind='cubic', axis=0)

    # Generate parameter values for the smooth curve
    t_smooth = np.linspace(0, 1, num_points+1)

    # Evaluate the cubic spline at the parameter values to generate the smooth curve
    curve = spline_func(t_smooth)

    # Extract x and y coordinates of the smooth curve
    x = curve[:, 0]
    y = curve[:, 1]

    # Return x and y coordinates of the closed curve
    return x, y


## Function to create Offset for Laminate coordinates

In [ ]:
def offset(x,y,offset):

    # calculate unit tangent vector at each point
    dx = np.gradient(x)
    dy = np.gradient(y)
    ds = np.sqrt(dx**2 + dy**2)
    tx = dx/ds
    ty = dy/ds

    # rotate unit tangent vector by 90 degrees to obtain unit normal vector
    nx = -ty
    ny = tx

    # scale unit normal vector by offset distance to obtain offset vector
    ox = nx * offset
    oy = ny * offset

    # translate each point along original curve by corresponding offset vector
    x_offset = x + ox
    y_offset = y + oy
    
    return x_offset, y_offset


In [ ]:
width = 20 * .0254
thickness = 2.5 * .0254
#bottom_curve = 1000
t_values = None
n = 10000
x,y = surfboard_outline(width,thickness,n)
plt.plot(x,y)
plt.axis('equal')


#### Carbon Stringer

###### Coordinates for all materials with the surfboard cross-section with a carbon stringer

In [ ]:

carbon_loc = 0 * .0254
top_lam_loc_y = -.25*.0254 + .5*.0254
bot_lam_loc_y = -.25*.0254 - .5*.0254

t = .25E-3
idx_bot_foam = (np.abs(y[0:int(n/2)] - carbon_loc)).argmin()
idx_top_foam = (np.abs(y[0:int(n/2)] - (carbon_loc+t))).argmin()
idx_top_lam = (np.abs(y[0:int(n/2)] - (top_lam_loc_y))).argmin()
idx_bot_lam = (np.abs(y[0:int(n/2)] - (bot_lam_loc_y))).argmin()

top_foam = [x[idx_top_foam:int(n/2)+(int(n/2)-idx_top_foam)],y[idx_top_foam:int(n/2)+(int(n/2)-idx_top_foam)]]

bot_foam = [np.append(x[0:idx_bot_foam],x[-idx_bot_foam:]),np.append(y[0:idx_bot_foam],y[-idx_bot_foam:])]

carbon = [np.append(x[idx_bot_foam:idx_top_foam],x[-idx_top_foam:-idx_bot_foam]),np.append(y[idx_bot_foam:idx_top_foam],y[-idx_top_foam:-idx_bot_foam])]

top_lam_bottom_layer = [x[idx_top_lam:int(n/2)+(int(n/2)-idx_top_lam)],y[idx_top_lam:int(n/2)+(int(n/2)-idx_top_lam)]]
top_lam_offset_x,top_lam_offset_y = offset(top_lam_bottom_layer[0],top_lam_bottom_layer[1],t*2)
top_lam = [np.append(top_lam_bottom_layer[0],top_lam_offset_x),np.append(top_lam_bottom_layer[1],top_lam_offset_y)] 

left_lam_bottom_layer = [x[idx_bot_lam:idx_top_lam],y[idx_bot_lam:idx_top_lam]]
left_lam_offset_x,left_lam_offset_y = offset(left_lam_bottom_layer[0],left_lam_bottom_layer[1],t*3)
left_lam = [np.append(left_lam_bottom_layer[0],left_lam_offset_x),np.append(left_lam_bottom_layer[1],left_lam_offset_y)] 

right_lam_bottom_layer = [x[-idx_top_lam:-idx_bot_lam],y[-idx_top_lam:-idx_bot_lam]]
right_lam_offset_x,right_lam_offset_y = offset(right_lam_bottom_layer[0],right_lam_bottom_layer[1],t*3)
right_lam = [np.append(right_lam_bottom_layer[0],right_lam_offset_x),np.append(right_lam_bottom_layer[1],right_lam_offset_y)] 

bot_lam_bottom_layer = [np.append(x[0:idx_bot_lam],x[-idx_bot_lam:]),np.append(y[0:idx_bot_lam],y[-idx_bot_lam:])]
bot_lam_offset_x,bot_lam_offset_y = offset(bot_lam_bottom_layer[0],bot_lam_bottom_layer[1],t*2)
bot_lam = [np.append(bot_lam_bottom_layer[0],bot_lam_offset_x),np.append(bot_lam_bottom_layer[1],bot_lam_offset_y)] 



#plt.plot([-.3,.3],[-.15*.0254,-.15*.0254],linestyle='--')
plt.scatter(top_foam[0],top_foam[1],s=.25,color='silver')
plt.scatter(bot_foam[0],bot_foam[1],s=.25,color='silver')
plt.scatter(top_lam[0],top_lam[1],s=.25,color='brown')
plt.scatter(left_lam[0],left_lam[1],s=.25,color='brown')
plt.scatter(right_lam[0],right_lam[1],s=.25,color='brown')
plt.scatter(bot_lam[0],bot_lam[1],s=.25,color='brown')

plt.plot(carbon[0],carbon[1],color='black')
plt.axis('equal')
